In [16]:
from keras.models import Model
from keras import layers
from keras import Input

text_max_words = 10000
answer_max_words = 200

### 잔차연결에서 중요한 것.
- layers.add함수인데, 여기에 hidden층 결과와 text_input을 더해주는데, 이건 단순히 행렬 덧셈이야.
- concatenate와는 달라. 이건 행렬을 연결해 주는 것이고.
- 그래서 단순 행렬 덧셈을 하려면, hidden층 결과와 text_input 층 결과의 형상(shape)이 같아야 한다. (batch_size, 특징수)로 같아야 한다.
- 여기서는 text_input shpae에 (None,)으로 특징수를 열어놓았지만, 실은 hidden층 LSTM(32)처럼 32이라고 못박아도 상관없다. 이게 나을듯. 나중 데이터 주입할 때, 더 헷갈릴 수 있음.

In [22]:
text_input = Input(shape=(32,), dtype='float32', name='text')
hidden = layers.Embedding(text_max_words, 64)(text_input)
hidden = layers.LSTM(32)(hidden)

hidden = layers.add([hidden, text_input])
print('hidden shape:', hidden.shape)
print('text_input shape:', text_input.shape)

hidden shape: (None, 32)
text_input shape: (None, 32)


In [23]:
result = layers.Dense(answer_max_words, activation='softmax')(hidden)

In [25]:
model = Model(inputs=text_input, outputs=result)

In [1]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

NameError: name 'model' is not defined

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 32, 64)       640000      text[0][0]                       
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (None, 32)           12416       embedding_11[0][0]               
__________________________________________________________________________________________________
add_10 (Add)                    (None, 32)           0           lstm_11[0][0]                    
                                                                 text[0][0]                 

In [28]:
from tensorflow.keras.utils import to_categorical
import numpy as np

num_samples = 10000
max_len = 32

inputs = np.random.randint(low=0, high=text_max_words, 
                          size=(num_samples, max_len))

result = np.random.randint(low=0, high=answer_max_words, size=num_samples)
result = to_categorical(result)

In [29]:
print('inputs.shape:', inputs.shape)
print('\nresult.shape:', result.shape)
print('\ninputs:', inputs)
print('\nresult:', result)

inputs.shape: (10000, 32)

result.shape: (10000, 200)

inputs: [[6906 9515 6226 ... 6718 8769 5473]
 [9212 3514 8950 ... 9282 5843  696]
 [5171 3671  905 ... 6311 7275 2674]
 ...
 [5379 5808  963 ...  225  880  559]
 [3084 5381 7166 ... 3480 4435 1768]
 [  64 9142 6416 ... 7900 2782 5956]]

result: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(inputs, result, epochs=3, batch_size=8)

/Users/jsha/opt/anaconda3/envs/nlp3710/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
